In [1]:
import pandas as pd
import numpy as np

In [2]:
import sklearn 

In [4]:
#url='https://raw.githubusercontent.com/skathirmani/datasets/master/HR%20Analytics.csv'
hr=pd.read_csv('HR Analytics.csv',sep=',')

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [6]:
hr_dummy=pd.get_dummies(hr)
train,test=train_test_split(hr_dummy,
                            test_size=0.3,
                            random_state=100)
print(train.shape)

(1029, 56)


In [7]:

train_y=train['Attrition']
test_y=test['Attrition']

train_x=train.drop('Attrition',axis=1)
test_x=test.drop('Attrition',axis=1)  ## the above is removing the attrition column as we need to predict that that shud be out output column so therefore removing it just remember it with linear regression y=mx+c 

train_x.shape


(1029, 55)

In [8]:
model=RandomForestClassifier(random_state=100)


In [9]:
model.fit(train_x,train_y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=100, verbose=0, warm_start=False)

In [10]:
test_pred=model.predict(test_x)

In [11]:
df_pred=pd.DataFrame({'actual': test_y,
                      'predicted':test_pred})


In [12]:
df_pred.head()


,actual,predicted
880,0,0
152,0,0
1466,0,0
1084,0,0
1086,0,0


In [13]:
df_pred['pred_status']=df_pred['actual']==df_pred['predicted']

In [14]:
df_pred

,actual,predicted,pred_status
880,0,0,True
152,0,0,True
1466,0,0,True
1084,0,0,True
1086,0,0,True
1392,0,0,True
57,0,0,True
956,0,0,True
1400,0,0,True
1175,0,0,True


In [15]:
## true positive
tp=df_pred[(df_pred['predicted']==1) & (df_pred['actual']==1)].shape[0]

In [16]:
tp

11

In [17]:
###true negative
tn=df_pred[(df_pred['predicted']==0) & (df_pred['actual']==0)].shape[0]

In [18]:
tn

367

In [19]:
##false positive
fp=df_pred[(df_pred['predicted']==1) & (df_pred['actual']==0)].shape[0]

In [20]:
fp

4

In [21]:
##false negative
fn=df_pred[(df_pred['predicted']==0) & (df_pred['actual']==1)].shape[0]

In [22]:
fn

59

In [23]:
### we can do that using sklearn as that function of the confusion matrix is present
from sklearn.metrics import confusion_matrix

In [23]:
confusion_matrix(df_pred['actual'],df_pred['predicted'])

array([[367,   4],
       [ 59,  11]], dtype=int64)

In [24]:
tn,fp,fn,tp =confusion_matrix(df_pred['actual'],df_pred['predicted']).ravel()

In [25]:
print(tn,fp,fn,tp)

367 4 59 11


In [26]:
from sklearn.metrics import classification_report

print(classification_report(df_pred['actual'], 
                            df_pred['predicted']))

             precision    recall  f1-score   support

          0       0.86      0.99      0.92       371
          1       0.73      0.16      0.26        70

avg / total       0.84      0.86      0.82       441



# Adaptive Boost (Ada Boost)

In [27]:
from sklearn.ensemble import AdaBoostClassifier

In [28]:
model = AdaBoostClassifier(random_state=100)
model.fit(train_x, train_y)

test_pred = model.predict(test_x)
tn, fp, fn, tp = confusion_matrix(test_y, test_pred).ravel()
tn, fp, fn, tp

(352, 19, 43, 27)

In [29]:
acc = (tp+tn)/(tp+tn+fp+fn)
acc

0.8594104308390023

In [30]:
sensitivity = tp / (tp + fn)
sensitivity

0.38571428571428573

# Weighted sampling

In [31]:
pd.Series(np.random.randint(1,100,100)).value_counts().shape

(66,)

In [32]:
random_samples = []
for i in range(100):
    sample = np.random.choice([1,2,3], p =[0,0.2,0.8])
    random_samples.append(sample)
pd.Series(random_samples).value_counts()

3    83
2    17
dtype: int64

# Classification Error Rate

In [33]:
samples_dt1 = train.iloc[np.random.randint(1,train.shape[0], train.shape[0])]
samples_dt1

all_cols = np.array(train.drop('Attrition', axis=1).columns)
cols_position = np.random.randint(1,len(all_cols) ,3)
random_cols = all_cols[cols_position]

samples_dt1_x = samples_dt1[random_cols]
samples_dt1_y = samples_dt1['Attrition']
random_cols

array(['YearsAtCompany', 'JobSatisfaction',
       'JobRole_Manufacturing Director'], dtype=object)

In [34]:
pd.Series(samples_dt1.index).value_counts().head()

853     5
1026    5
274     5
1081    5
976     5
dtype: int64

In [35]:
dt1 = DecisionTreeClassifier(max_depth=1)

dt1.fit(samples_dt1_x, samples_dt1_y)

test_pred = dt1.predict(test_x[random_cols])

df_pred = pd.DataFrame({'actual': test_y,
                       'predicted': test_pred})
df_pred.head()

,actual,predicted
880,0,0
152,0,0
1466,0,0
1084,0,0
1086,0,0


In [36]:
from sklearn.metrics import confusion_matrix
tn, fp, fn,tp = confusion_matrix(test_y, test_pred).ravel()
tn, fp, fn, tp

(371, 0, 70, 0)

In [37]:
test_y.shape

(441,)

In [38]:
import math

In [39]:
# Error rate is the false predictions divided by total predictions
# no of misclassifications upon total predictions

Error_rate = (fp+fn)/(tn+tp+fn+fp)
Error_rate

0.15873015873015872

In [40]:
#alpha_t is the weightage of the tree

alpha_t= 0.5 * math.log((1-Error_rate)/Error_rate)
alpha_t

0.8338534102790381